# Project - Logical Architecture Elements Study 

This is a study that will use properties in the a Capella model to perform a study on the number of solar panels required to satisfy various a few usage scenarios.

Access the System Model that is in GitHUB folder with a python module "caplellambse" that can read and modify the SMW/Capella model content.

In [1]:
import capellambse
import ipywidgets as widgets
from IPython.core.display import HTML
import numpy as np
import pandas as pd 
from IPython.display import Markdown



resources = {
    "Diesel_Engine": "Diesel_Engine/Diesel_Engine",
}
path_to_model = "../Diesel_Engine.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)
model = capellambse.MelodyModel(path_to_model, resources=resources)

Display a diagram from the capella module with parameters being accessed. Note that this diagram is not being pulled dynamically, due to the desire to see the parameters on the diagram. It is pulling a snapshot of a .jpg diagram.

In [2]:
#from IPython import display as diag_display
#diag_display.Image("../Images/[LAB] Trail Power (Charge) Properties.jpg")


## Display of all the parameters and respective components/functions in the logical architecture
The following is a report of all the parameters avialable to be used from the logical architecture. 

In [3]:
la_model = model.la 

df = pd.DataFrame({
    'Logical Function': [],
    'Property Value Group Name': [],
    'Propery Name': [],
    'Property Value': [],
    })

for function in la_model.all_functions:
    for pvg in  function.applied_property_value_groups :
        for pv in  pvg.property_values:
            #print("LogicalFunction=",'"' +function.name+ '"',"property_value_groups=",'"' +pvg.name+ '"','"Property Name"=','"' +pv.name+ '"','"Value"=',pv.value )
            df.loc[len(df)] = [function.name,pvg.name,\
                                pv.name, pv.value]
display(df)
df = pd.DataFrame({
    'Logical Component': [],
    'Property Value Group Name': [],
    'Propery Name': [],
    'Property Value': [],
    })
for component in la_model.all_components:
    for pvg in  component.applied_property_value_groups :
        for pv in  pvg.property_values:
            #print("LogicalComponents=",'"' +component.name+ '"',"property_value_groups=",'"' +pvg.name+ '"','Property Name=','"' + pv.name + '"','Value=',pv.value  )
            df.loc[len(df)] = [component.name,pvg.name,\
                                pv.name, pv.value]
display(df)

,Logical Function,Property Value Group Name,Propery Name,Property Value


,Logical Component,Property Value Group Name,Propery Name,Property Value


The following code can be used in the study to retrive the value from the model to use in the study.

your_variable = model.search("LogicalFunction").by_name("Name of Function").property_value_groups["Name of property Value Group"]["Name of Property"])

your_variable = model.search("LogicalComponent").by_name("Name of Component").property_value_groups["Name of property Value Group"]["Name of Property"])

You can see an example below.

In [4]:
#print( "Battery Size" , model.search("LogicalComponent").by_name("Battery").property_value_groups["Battery Capacity"]["Value"] ,\
#"Units:" ,model.search("LogicalComponent").by_name("Battery").property_value_groups["Battery Capacity"]["Units"])

# Analysis by Functional Chain
Typically analysis could be done by functional chain. You may even want leverage content of the model to define variable in your study.

In [5]:


# Currently it does not handle Chains that Reference Chains. 
# Looking at options on how to handle it. 
for fc in model.la.all_functional_chains:
    #print(fc)
    #print()
    #print("Functional Chain:", fc.name)
    display(Markdown(f"# Functional Chain: {fc.name}"))


    df = pd.DataFrame({
    'Function Owner': [],
    'Function': [],
    'Function Exchange': [],
    'Function Exchange Items': [],
    'Function Exchange Item Elements': [],
    })
    def Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                               Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE) :
        df.loc[len(df)] = [Current_Function_Owner ,Current_Function,\
                                Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE]
        
    for function in fc.involved_functions:
        #This will initialize all the table elements
        Current_Function_Owner = ''
        Current_Function =''
        Current_Function_Exchange = ''
        Current_Function_Exchange_EI = ''
        Current_Function_Exchange_EIE = ''
        
        #print(function.owner)
        if function.owner != None :
         
            Current_Function_Owner=function.owner.name
            #print("- Entity:",Current_Function_Owner)
            #print("-Function:",function.name)
            Current_Function = function.name
            #print("-Function:",Current_Function)
           
            
            #print("---Exch:",exchange.name)
            for output in function.outputs:
               # print(output)
                is_FOs= True
                Current_Function_OPort = output.name
                #Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                #                 Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)
                for exchange in output.exchanges:         
                    for involved_exchange in fc.involved_links:
                         if exchange == involved_exchange:
                             Current_Function_Exchange = exchange.name
                             #print("--Output:",output.name)
                             #print("---Exch:",Current_Function_Exchange)
                             
                             for exchange_item in exchange.exchange_items: 
                                 Current_Function_Exchange_EI = exchange_item.name
                                 #print("--Output:",output.name)
                                 #print(exchange_item)
                                 #print("---Exch Item:",Current_Function_Exchange_EI)

                                 for element in exchange_item.elements: 
                                     Current_Function_Exchange_EIE = element.name
                                     #print(element)
                                     #print("---Exch Item Elements:",Current_Function_Exchange_EIE)
                                     Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                        Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)
                                 if  exchange_item.elements ==  []:
                                     Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                    Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE) 
                             if exchange.exchange_items == [] :
                                Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE) 
          
        else: 
            Current_Function_Owner="None"
            print("-Entity","None")
            Current_Function = function.name
            print("-Function",Current_Function )

            for output in function.outputs:  
                Current_Function_OPort = output.name
                Current_Function_OPort = output.name
                #Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                #    Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)
                for exchange in output.exchanges:
                   for involved_exchange in fc.involved_links:
                         if exchange == involved_exchange:
                             
                             
                             Current_Function_Exchange = exchange.name
                             #print("--Output:",output.name)
                             #print("---Exch:",Current_Function_Exchange)
                             if exchange.exchange_items == [] :
                                 Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                 Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)   
                             for exchange_item in exchange.exchange_items: 
                                 Current_Function_Exchange_EI = exchange_item.name
                                 #print("--Output:",output.name)
                                 #print(exchange_item)
                                 #print("---Exch Item:",Current_Function_Exchange_EI)
                                 
                                 
                                 for element in exchange_item.elements: 
                                     Current_Function_Exchange_EIE = element.name
                                     #print(element)
                                     #print("---Exch Item Elements:",Current_Function_Exchange_EIE)
                                     Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                       Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)
                                 if exchange_item.elements == []:
                                     Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                    Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE) 
                             if exchange.exchange_items == [] :
                                 Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                                 Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)    
                               
            if  function.outputs == []:
                Append_Row_to_Dataframe(df,  Current_Function_Owner ,Current_Function,\
                     Current_Function_Exchange , Current_Function_Exchange_EI, Current_Function_Exchange_EIE)                                 

    

    display(df)
        


# Functional Chain: Provide responsive rotational torque

-Entity None
-Function Deliver rotational torque proportional to power setting


,Function Owner,Function,Function Exchange,Function Exchange Items,Function Exchange Item Elements
0,None,Deliver rotational torque proportional to powe...,,,
1,Opearator,Provide Trottle Position,Throttle Angle,,
2,Fuel,Provide Fuel,Fuel,,
3,Engine Control Unit,"Controls fuel injection timing, air intake, an...",FunctionalExchange 16,,
4,Engine Control Unit,"Controls fuel injection timing, air intake, an...",FunctionalExchange 3,,
5,Crankshaft,Converts linear motion of pistons into rotatio...,FunctionalExchange 9,,
6,Crankshaft,Converts linear motion of pistons into rotatio...,Rotational Torqur,,
7,Fuel Injectors,Delivers diesel fuel into the combustion chamber,FunctionalExchange 7,,
8,Fuel Pump,Pumps diesel fuel from tank to injectors at hi...,FunctionalExchange 1,,
9,Cylinders,Chambers where air is compressed and diesel fu...,FunctionalExchange 18,,


# Functional Chain: Acchieve Low Emissions

,Function Owner,Function,Function Exchange,Function Exchange Items,Function Exchange Item Elements
0,Particulate Filter,Captures soot and particulate matter from exha...,FunctionalExchange 27,,
1,Particulate Filter,Captures soot and particulate matter from exha...,Emissions,,
2,Exhaust Manifold,Collects exhaust gases and directs them to the...,FunctionalExchange 24,,


...

,Function Owner,Function,Function Exchange,Function Exchange Items,Function Exchange Item Elements
0,None,Deliver rotational torque proportional to powe...,,,
1,Radiator,Cools the engine by dissipating heat from the ...,Heat,,


# Functional Chain: Minimize Vibration

,Function Owner,Function,Function Exchange,Function Exchange Items,Function Exchange Item Elements
0,Crankshaft,Converts linear motion of pistons into rotatio...,FunctionalExchange 10,,
1,Crankshaft,Converts linear motion of pistons into rotatio...,Vibration,,


# Functional Chain: Provide Emission Economy Modes

,Function Owner,Function,Function Exchange,Function Exchange Items,Function Exchange Item Elements
0,Opearator,Provide Fuel Econonmy/Emission Setting,Operation Mode,,
